# YouTube Crawling 

In [1]:
import pandas as pd
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option('display.max.colwidth',30)
pd.options.display.max_columns = 100

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
from bs4 import BeautifulSoup
import re

In [218]:
driver = webdriver.Chrome('chromedriver.exe')
driver.maximize_window()

# YouTuber 1 - 요남 TV

### 영상  정보 수집 ( 제목 / url * / 업로드 날짜 / 조회수 / 좋아요 / 싫어요 )
- 전체 영상 목록이 보이는 페이지에서 각 동영상의 url 추출하는 과정

In [241]:
# 원하는 유튜버 '동영상'에 들어가기 
driver = webdriver.Chrome('chromedriver.exe')
driver.get('https://www.youtube.com/channel/UCyozK5OFN5lDrwim5wqQnLA/videos')
body = driver.find_element_by_tag_name('body')
time.sleep(1.5)#브라우저 로딩시간기다려야함

# 유튜브는 스크롤해야 전체를 볼 수 있음 
endk = 5 # 5번 내림
while endk:
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(0.3)
    endk -= 1    
start_page = driver.page_source 
soup = BeautifulSoup(start_page,'lxml') # 전체 영상 목록의 page-source 이니 soup으로

# 각 영상 페이지의 url 모음
video_list = soup.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

YN_url = []
base_url = 'http://www.youtube.com'
for i in range(len(video_list)):
    url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']
    YN_url.append(url)

- 추출한 모든 영상의 url을 입력하여, 각 페이지 안으로 들어가 정보를 수집

In [242]:
YN = pd.DataFrame({'youtuber' :[],
                   'subscribers' : [],
                   'name':[],
#                    'video_url':[],
                   'upload_date':[],
                   'hits' :[],
                   'likes_num':[],
                   'dislikes_num':[]})

for i in range(0,len(YN_url)):
    
    youtuber = soup.find('title').text
    sub_num = soup.find('yt-formatted-string',{'id':'subscriber-count'}).text
    name = video_list[i].find('a',{'id':'video-title'}).text
#     url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']
    

    # 각 동영상 페이지 들어가기 시작 
    start_url = YN_url[i]
    driver.get(start_url) 
    body = driver.find_element_by_tag_name('body')
    
    time.sleep(1.5)
    
    # 댓글까지 쭉 보려면 스크롤
    endk = 5 # 5번 내림
    while endk:
        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        time.sleep(0.3) # 내리는 시간 간격 
        endk -= 1    
    page = driver.page_source 
    html = BeautifulSoup(page,'lxml') # 각 페이지의 page_source 이니 html로
    
    
    # 각 동영상 페이지 들어가서 얻을 수 있는 정보
    pre_upload_date = html.find('div',{'id':'date'})
    upload_date = pre_upload_date.find('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'}).text   
    pre_hits = html.find('yt-view-count-renderer', {'class':'style-scope ytd-video-primary-info-renderer'})
    hits = pre_hits.find('span',{'class':'view-count style-scope yt-view-count-renderer'}).text.split()[1]
    likes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('좋아요')}).text+'개'
    dislikes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('싫어요')}).text+'개'

    # 마지막에 모은 정보 합치기
    insert_data = pd.DataFrame({'youtuber' :[youtuber],
                                'subscribers' : [sub_num],
                                'name':[name],
#                                 'video_url':[url],
                                'upload_date':[upload_date], 
                                'hits' : [hits],
                                'likes_num':[likes_num],
                                'dislikes_num':[dislikes_num]})
    
    YN = YN.append(insert_data)

- 수집된 데이터 약간의 전처리 

In [243]:
YN  

,youtuber,subscribers,name,upload_date,hits,likes_num,dislikes_num
0,취미로 요리하는 남자 Yonam - YouTube,구독자 55.6만명,요리사들이 한 자리에 모이면 벌어지는 일 2 (...,최초 공개: 2020. 3. 25.,"245,137회",5.8천개,67개
0,취미로 요리하는 남자 Yonam - YouTube,구독자 55.6만명,sub)요리하는 남자의 간단한 집들이음식,최초 공개: 2020. 3. 4.,"630,196회",9.5천개,124개
0,취미로 요리하는 남자 Yonam - YouTube,구독자 55.6만명,(종료)제가 여러분께 요리를 해드립니다.,2020. 2. 20.,"160,997회",3.4천개,21개
0,취미로 요리하는 남자 Yonam - YouTube,구독자 55.6만명,sub)요리사들이 한 자리에 모이면 벌어지는 일.,최초 공개: 2020. 2. 4.,"1,962,881회",2.1만개,384개
0,취미로 요리하는 남자 Yonam - YouTube,구독자 55.6만명,sub)요리하는 남자들의 흔한 파티음식.,최초 공개: 2020. 1. 23.,"720,225회",9천개,144개
0,취미로 요리하는 남자 Yonam - YouTube,구독자 55.6만명,"ENG)요리하는 남자의 글램핑, 스테이크, 비프...",최초 공개: 2019. 12. 5.,"477,349회",5.4천개,74개
0,취미로 요리하는 남자 Yonam - YouTube,구독자 55.6만명,취미로 요리하는 남자가 무려 다섯명. feat....,최초 공개: 2019. 11. 26.,"696,346회",6.9천개,129개
0,취미로 요리하는 남자 Yonam - YouTube,구독자 55.6만명,SUB) 바닷가에서 먹는 브런치...는 개뿔.,최초 공개: 2019. 11. 14.,"411,963회",5천개,82개
0,취미로 요리하는 남자 Yonam - YouTube,구독자 55.6만명,SUB) 요리하는 남자가 캠핑카를 타면 벌어지는 일.,최초 공개: 2019. 10. 29.,"556,207회",7천개,111개
0,취미로 요리하는 남자 Yonam - YouTube,구독자 55.6만명,SUB) 요리하는 남자. 캠핑카. 그리고 랍스터.,최초 공개: 2019. 10. 20.,"808,580회",9.5천개,211개


In [244]:
YN.to_csv('YN.csv', encoding='utf-8-sig')

# YouTueber2 - 승우아빠

In [227]:
# 원하는 유튜버 '동영상'에 들어가기 
driver = webdriver.Chrome('chromedriver.exe')
driver.get('https://www.youtube.com/channel/UCgsffS7MfKL6YU3r_U3E-aA/videos')
body = driver.find_element_by_tag_name('body')
time.sleep(1.5)#브라우저 로딩시간기다려야함

# 유튜브는 스크롤해야 전체를 볼 수 있음 
endk = 30 # 승우아빠는 영상이 많아서 넉넉히 많이 내림
while endk:
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(0.3)
    endk -= 1    
start_page = driver.page_source 
soup = BeautifulSoup(start_page,'lxml') # 전체 영상 목록의 page-source 이니 soup으로

# 각 영상 페이지의 url 모음
video_list = soup.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

SW_url = []
base_url = 'http://www.youtube.com'
for i in range(len(video_list)):
    url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']
    SW_url.append(url)

In [228]:
len(SW_url)

366

In [232]:
SW = pd.DataFrame({'youtuber' :[],
                   'subscribers' : [],
                   'name':[],
#                    'video_url':[],
                   'upload_date':[],
                   'hits' :[],
                   'likes_num':[],
                   'dislikes_num':[]})

for i in range(0,len(SW_url)):
    
    youtuber = soup.find('title').text
    sub_num = soup.find('yt-formatted-string',{'id':'subscriber-count'}).text
    name = video_list[i].find('a',{'id':'video-title'}).text
#     url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']
    

    # 각 동영상 페이지 들어가기 시작 
    start_url = SW_url[i]
    driver.get(start_url) 
    body = driver.find_element_by_tag_name('body')
    
    time.sleep(1.5)
    
    # 댓글까지 쭉 보려면 스크롤
    endk = 5 # 5번 내림
    while endk:
        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        time.sleep(0.3) # 내리는 시간 간격 
        endk -= 1    
    page = driver.page_source 
    html = BeautifulSoup(page,'lxml') # 각 페이지의 page_source 이니 html로
    
    
    # 각 동영상 페이지 들어가서 얻을 수 있는 정보
    pre_upload_date = html.find('div',{'id':'date'})
    upload_date = pre_upload_date.find('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'}).text   
    pre_hits = html.find('yt-view-count-renderer', {'class':'style-scope ytd-video-primary-info-renderer'})
    hits = pre_hits.find('span',{'class':'view-count style-scope yt-view-count-renderer'}).text.split()[1]
    likes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('좋아요')}).text+'개'
    dislikes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('싫어요')}).text+'개'

    # 마지막에 모은 정보 합치기
    insert_data = pd.DataFrame({'youtuber' :[youtuber],
                                'subscribers' : [sub_num],
                                'name':[name],
#                                 'video_url':[url],
                                'upload_date':[upload_date], 
                                'hits' : [hits],
                                'likes_num':[likes_num],
                                'dislikes_num':[dislikes_num]})
    
    SW = SW.append(insert_data)

In [233]:
SW

,youtuber,subscribers,name,upload_date,hits,likes_num,dislikes_num
0,승우아빠 - YouTube,구독자 44.2만명,51시간동안 바베큐한 등갈비,4시간 전 최초 공개,"29,770회",1.5천개,14개
0,승우아빠 - YouTube,구독자 44.2만명,음식 무조건 맛있게 하는 공식 알려드립니다,2020. 3. 25.,"142,136회",4.2천개,41개
0,승우아빠 - YouTube,구독자 44.2만명,굴소스는 사드세요.....제발,2020. 3. 23.,"410,497회",5.1천개,96개
0,승우아빠 - YouTube,구독자 44.2만명,세상 간단한 마이야르 치킨 3가지,최초 공개: 2020. 3. 20.,"180,382회",3.6천개,52개
0,승우아빠 - YouTube,구독자 44.2만명,절대로 집에서 요리하면 안되는 사람들,2020. 3. 18.,"208,209회",5.7천개,71개
0,승우아빠 - YouTube,구독자 44.2만명,세상 간단한 홍게요리 4가지와 게살버거,2020. 3. 16.,"160,119회",2.4천개,49개
0,승우아빠 - YouTube,구독자 44.2만명,세상 간단한 통조림 요리 7가지,최초 공개: 2020. 3. 13.,"146,496회",3.1천개,53개
0,승우아빠 - YouTube,구독자 44.2만명,달걀60개로 김계란 만들기,2020. 3. 11.,"345,845회",3.6천개,91개
0,승우아빠 - YouTube,구독자 44.2만명,400번 떨궈만드는 짜고나 국수를 만들었,최초 공개: 2020. 3. 9.,"195,381회",3.2천개,59개
0,승우아빠 - YouTube,구독자 44.2만명,24시간동안 우대갈비를 요리해 보았습니다,최초 공개: 2020. 3. 6.,"293,449회",3.5천개,112개


In [239]:
SW.to_csv('SW.csv', encoding='utf-8-sig')

# YouTuber 3 - 먹어볼래

In [245]:
# 원하는 유튜버 '동영상'에 들어가기 
driver = webdriver.Chrome('chromedriver.exe')
driver.get('https://www.youtube.com/channel/UCTx3aCntDvkq-hGtOjKVSnQ/videos')
body = driver.find_element_by_tag_name('body')
driver.maximize_window()
time.sleep(1.5)#브라우저 로딩시간기다려야함

# 유튜브는 스크롤해야 전체를 볼 수 있음 
endk = 30 
while endk:
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(0.3)
    endk -= 1    
start_page = driver.page_source 
soup = BeautifulSoup(start_page,'lxml') # 전체 영상 목록의 page-source 이니 soup으로

# 각 영상 페이지의 url 모음
video_list = soup.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

MB_url = []
base_url = 'http://www.youtube.com'
for i in range(len(video_list)):
    url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']
    MB_url.append(url)

In [246]:
len(MB_url)

84

In [247]:
MB = pd.DataFrame({'youtuber' :[],
                   'subscribers' : [],
                   'name':[],
#                    'video_url':[],
                   'upload_date':[],
                   'hits' :[],
                   'likes_num':[],
                   'dislikes_num':[]})

for i in range(0,len(MB_url)):
    
    youtuber = soup.find('title').text
    sub_num = soup.find('yt-formatted-string',{'id':'subscriber-count'}).text
    name = video_list[i].find('a',{'id':'video-title'}).text
#     url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']
    

    # 각 동영상 페이지 들어가기 시작 
    start_url = MB_url[i]
    driver.get(start_url) 
    body = driver.find_element_by_tag_name('body')
    
    time.sleep(1.5)
    
    # 댓글까지 쭉 보려면 스크롤
    endk = 5 # 5번 내림
    while endk:
        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        time.sleep(0.3) # 내리는 시간 간격 
        endk -= 1    
    page = driver.page_source 
    html = BeautifulSoup(page,'lxml') # 각 페이지의 page_source 이니 html로
    
    
    # 각 동영상 페이지 들어가서 얻을 수 있는 정보
    pre_upload_date = html.find('div',{'id':'date'})
    upload_date = pre_upload_date.find('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'}).text   
    pre_hits = html.find('yt-view-count-renderer', {'class':'style-scope ytd-video-primary-info-renderer'})
    hits = pre_hits.find('span',{'class':'view-count style-scope yt-view-count-renderer'}).text.split()[1]
    likes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('좋아요')}).text+'개'
    dislikes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('싫어요')}).text+'개'

    # 마지막에 모은 정보 합치기
    insert_data = pd.DataFrame({'youtuber' :[youtuber],
                                'subscribers' : [sub_num],
                                'name':[name],
#                                 'video_url':[url],
                                'upload_date':[upload_date], 
                                'hits' : [hits],
                                'likes_num':[likes_num],
                                'dislikes_num':[dislikes_num]})
    
    MB = MB.append(insert_data)

In [248]:
MB

,youtuber,subscribers,name,upload_date,hits,likes_num,dislikes_num
0,먹어볼래TryToEat - YouTube,구독자 60.9만명,갈비에 미친 사람이 뼈해장국을 끓이면,2020. 3. 26.,"124,537회",2.1천개,59개
0,먹어볼래TryToEat - YouTube,구독자 60.9만명,육회에 미친 사람이 햄버거를 만들면,2020. 3. 21.,"567,403회",6천개,216개
0,먹어볼래TryToEat - YouTube,구독자 60.9만명,진짜 벌집으로 피자를 만들면! 벌집핏자,2020. 3. 18.,"416,951회",4.9천개,193개
0,먹어볼래TryToEat - YouTube,구독자 60.9만명,통순대로 돈까스를 만들면,2020. 3. 14.,"252,636회",3.1천개,135개
0,먹어볼래TryToEat - YouTube,구독자 60.9만명,라면 소스로 요식업계 정점! 불범벅 클라쓰,2020. 3. 12.,"295,493회",3.4천개,128개
0,먹어볼래TryToEat - YouTube,구독자 60.9만명,찢어먹는 삼겹살! 이름하야 찢겹살,2020. 3. 8.,"313,813회",4천개,169개
0,먹어볼래TryToEat - YouTube,구독자 60.9만명,라면을 조합하는 새로운 방법! 산더미 앵그리 짜파구리,2020. 3. 6.,"292,751회",3.9천개,134개
0,먹어볼래TryToEat - YouTube,구독자 60.9만명,서브웨이에서 파스타를 만들면,2020. 2. 29.,"406,774회",4.9천개,190개
0,먹어볼래TryToEat - YouTube,구독자 60.9만명,게 없이 간장게장맛 만드는 법,2020. 2. 22.,"347,070회",4.4천개,222개
0,먹어볼래TryToEat - YouTube,구독자 60.9만명,통삼겹 통연어로 짜장면 만드는 법! 욕망의 통통...,2020. 2. 15.,"597,212회",1.1만개,425개


In [249]:
MB.to_csv('MB.csv',encoding='utf-8-sig')

# Youtuber 4 - 만개의 레시피
- 2991개만 가능

In [3]:
# 원하는 유튜버 '동영상'에 들어가기 
driver = webdriver.Chrome('chromedriver.exe')
driver.get('https://www.youtube.com/channel/UCKA_6r3CWC76x_EaFO6jsPA/videos')
body = driver.find_element_by_tag_name('body')
driver.maximize_window()
time.sleep(1.5)#브라우저 로딩시간기다려야함

In [13]:
# 유튜브는 스크롤해야 전체를 볼 수 있음 
endk = 400
while endk:
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(0.3)
    endk -= 1   
    
start_page = driver.page_source 
soup = BeautifulSoup(start_page,'lxml') # 전체 영상 목록의 page-source 이니 soup으로

# 각 영상 페이지의 url 모음
video_list = soup.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

MR_url= []
base_url = 'http://www.youtube.com'
for i in range(len(video_list)):
    url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']
    MR_url.append(url)

In [15]:
len(MR_url)

2991

In [18]:
MR = pd.DataFrame({'youtuber' :[],
                   'subscribers' : [],
                   'name':[],
#                    'video_url':[],
                   'upload_date':[],
                   'hits' :[],
                   'likes_num':[],
                   'dislikes_num':[]})

for i in range(0,len(MR_url)):
    
    try :
        youtuber = soup.find('title').text
        sub_num = soup.find('yt-formatted-string',{'id':'subscriber-count'}).text
        name = video_list[i].find('a',{'id':'video-title'}).text
    #     url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']


        # 각 동영상 페이지 들어가기 시작 
        start_url = MR_url[i]
        driver.get(start_url) 
        body = driver.find_element_by_tag_name('body')

        time.sleep(1.5)

        # 댓글까지 쭉 보려면 스크롤
        endk = 5 # 5번 내림
        while endk:
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(0.3) # 내리는 시간 간격 
            endk -= 1    
        page = driver.page_source 
        html = BeautifulSoup(page,'lxml') # 각 페이지의 page_source 이니 html로


        # 각 동영상 페이지 들어가서 얻을 수 있는 정보
        pre_upload_date = html.find('div',{'id':'date'})
        upload_date = pre_upload_date.find('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'}).text   
        pre_hits = html.find('yt-view-count-renderer', {'class':'style-scope ytd-video-primary-info-renderer'})
        hits = pre_hits.find('span',{'class':'view-count style-scope yt-view-count-renderer'}).text.split()[1]
        likes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('좋아요')}).text+'개'
        dislikes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('싫어요')}).text+'개'
        
    
        # 마지막에 모은 정보 합치기
        insert_data = pd.DataFrame({'youtuber' :[youtuber],
                                    'subscribers' : [sub_num],
                                    'name':[name],
    #                                 'video_url':[url],
                                    'upload_date':[upload_date], 
                                    'hits' : [hits],
                                    'likes_num':[likes_num],
                                    'dislikes_num':[dislikes_num]})

        MR = MR.append(insert_data)
    
    except AttributeError as e: 
        print(e)

'NoneType' object has no attribute 'text'


In [21]:
MR

,youtuber,subscribers,name,upload_date,hits,likes_num,dislikes_num
0,만개의레시피 10K Recipe - YouTube,구독자 35.6만명,봄이 왔나 봄! 봄나물로 만드는 봄채소 레시피 ...,2020. 3. 29.,410회,41개,0개
0,만개의레시피 10K Recipe - YouTube,구독자 35.6만명,이젠 디저트도 편의점재료로 만든다! 부드러운 연...,2020. 3. 29.,"1,713회",77개,1개
0,만개의레시피 10K Recipe - YouTube,구독자 35.6만명,실패없이 수제리코타치즈 만들기 에어프라이어로 치...,2020. 3. 28.,"3,317회",122개,0개
0,만개의레시피 10K Recipe - YouTube,구독자 35.6만명,원하는 재료를 넣어 만드는 달걀요리는? ! 다양...,2020. 3. 28.,"3,729회",70개,3개
0,만개의레시피 10K Recipe - YouTube,구독자 35.6만명,당신은 민초단 입니까? 홈베이킹 민초디저트! 민...,2020. 3. 28.,"2,487회",75개,5개
0,만개의레시피 10K Recipe - YouTube,구독자 35.6만명,스테이크 빠네샌드위치 : 빠네빵으로 스테이크샌드...,2020. 3. 27.,"2,120회",39개,1개
0,만개의레시피 10K Recipe - YouTube,구독자 35.6만명,값비싼 딸기부페는 필요없어! 나만의 딸기뷔페 만...,2020. 3. 27.,"3,379회",99개,0개
0,만개의레시피 10K Recipe - YouTube,구독자 35.6만명,두부라자냐 같은 떠먹는피자 만들기! 두부피자 :...,2020. 3. 27.,"4,380회",206개,1개
0,만개의레시피 10K Recipe - YouTube,구독자 35.6만명,떡볶이의 업그레이드 버전! 해물가득! 단짠의 맛...,2020. 3. 26.,"1,780회",69개,1개
0,만개의레시피 10K Recipe - YouTube,구독자 35.6만명,전국 유명한 음식들을 따라한다! 음식으로 여행어...,2020. 3. 26.,"3,090회",66개,1개


In [20]:
MR.to_csv('MR.csv',encoding='utf-8-sig')